In [2]:
library(tidyverse)
library(testthat)
library(digest)
library(repr)
library(tidymodels)
library(cowplot)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘testthat’


The following object is masked from ‘package:dplyr’:

    matches


The following object is masked from ‘package:purrr’:

    is_null


The following object is masked from ‘package:tidyr’:

    matches


Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────

In [8]:
url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/00381/PRSA_data_2010.1.1-2014.12.31.csv"
pollution <- read_csv(url)
pollution
pollution[is.na(pollution)] <- 0
pollution <- mutate(pollution, HUMID = 100 - 5 * (TEMP - DEWP))
head(pollution)

Parsed with column specification:
cols(
  No = col_double(),
  year = col_double(),
  month = col_double(),
  day = col_double(),
  hour = col_double(),
  pm2.5 = col_double(),
  DEWP = col_double(),
  TEMP = col_double(),
  PRES = col_double(),
  cbwd = col_character(),
  Iws = col_double(),
  Is = col_double(),
  Ir = col_double()
)



No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
1,2010,1,1,0,NA,-21,-11,1021,NW,1.79,0,0
2,2010,1,1,1,NA,-21,-12,1020,NW,4.92,0,0
3,2010,1,1,2,NA,-21,-11,1019,NW,6.71,0,0
4,2010,1,1,3,NA,-21,-14,1019,NW,9.84,0,0
5,2010,1,1,4,NA,-20,-12,1018,NW,12.97,0,0
6,2010,1,1,5,NA,-19,-10,1017,NW,16.10,0,0
7,2010,1,1,6,NA,-19,-9,1017,NW,19.23,0,0
8,2010,1,1,7,NA,-19,-9,1017,NW,21.02,0,0
9,2010,1,1,8,NA,-19,-9,1017,NW,24.15,0,0


No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir,HUMID
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,2010,1,1,0,0,-21,-11,1021,NW,1.79,0,0,50
2,2010,1,1,1,0,-21,-12,1020,NW,4.92,0,0,55
3,2010,1,1,2,0,-21,-11,1019,NW,6.71,0,0,50
4,2010,1,1,3,0,-21,-14,1019,NW,9.84,0,0,65
5,2010,1,1,4,0,-20,-12,1018,NW,12.97,0,0,60
6,2010,1,1,5,0,-19,-10,1017,NW,16.10,0,0,55


In [9]:
set.seed(9999)

In [10]:
pollution_split <- initial_split(pollution, prop = 3/4, strata = pm2.5)
pollution_training <- training(pollution_split)
pollution_testing <- testing(pollution_split)

pollution_split
pollution_training
pollution_testing

<Analysis/Assess/Total>
<32869/10955/43824>

No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir,HUMID
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,2010,1,1,0,0,-21,-11,1021,NW,1.79,0,0,50
3,2010,1,1,2,0,-21,-11,1019,NW,6.71,0,0,50
4,2010,1,1,3,0,-21,-14,1019,NW,9.84,0,0,65
5,2010,1,1,4,0,-20,-12,1018,NW,12.97,0,0,60
6,2010,1,1,5,0,-19,-10,1017,NW,16.10,0,0,55
7,2010,1,1,6,0,-19,-9,1017,NW,19.23,0,0,50
13,2010,1,1,12,0,-19,-5,1015,NW,37.56,0,0,30
14,2010,1,1,13,0,-18,-3,1015,NW,40.69,0,0,25
15,2010,1,1,14,0,-18,-2,1014,NW,43.82,0,0,20


No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir,HUMID
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
2,2010,1,1,1,0,-21,-12,1020,NW,4.92,0,0,55
8,2010,1,1,7,0,-19,-9,1017,NW,21.02,0,0,50
9,2010,1,1,8,0,-19,-9,1017,NW,24.15,0,0,50
10,2010,1,1,9,0,-20,-8,1017,NW,27.28,0,0,40
11,2010,1,1,10,0,-19,-7,1017,NW,31.30,0,0,40
12,2010,1,1,11,0,-18,-5,1017,NW,34.43,0,0,35
17,2010,1,1,16,0,-19,-2,1015,NW,1.79,0,0,15
29,2010,1,2,4,138,-7,-5,1022,SE,6.25,2,0,90
36,2010,1,2,11,152,-8,-5,1026,SE,20.56,0,0,85


In [12]:
pollution_sumtrain <- pollution_training %>%
    group_by(year, month) %>%
    summarise(meanpm2.5 = mean(pm2.5), meanTEMP = mean(TEMP), meanIws = mean(Iws), meanHUMID = mean(HUMID))
pollution_sumtrain

pollution_sumtrainyear <- pollution_training %>%
    group_by(year) %>%
    summarise(meanpm2.5 = mean(pm2.5), meanTEMP = mean(TEMP), meanIws = mean(Iws), meanHUMID = mean(HUMID))
pollution_sumtrainyear

pollution_sumtrainmonth <- pollution_training %>%
    group_by(month) %>%
    summarise(meanpm2.5 = mean(pm2.5), meanTEMP = mean(TEMP), meanIws = mean(Iws), meanHUMID = mean(HUMID))
pollution_sumtrainmonth

`summarise()` regrouping output by 'year' (override with `.groups` argument)



year,month,meanpm2.5,meanTEMP,meanIws,meanHUMID
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2010,1,77.84053,-6.1125704,41.215816,45.609756
2010,2,97.08644,-1.9960707,13.898978,43.398821
2010,3,92.34552,3.4223035,23.210073,43.272395
2010,4,77.57065,10.8170290,61.197065,28.306159
2010,5,86.68739,20.9822380,22.396572,33.685613
2010,6,85.87774,24.3467153,10.198996,56.669708
2010,7,125.40472,27.6497278,14.318385,66.206897
2010,8,90.87156,25.6201835,15.339229,64.807339
2010,9,80.28143,20.1013133,13.432983,65.525328


`summarise()` ungrouping output (override with `.groups` argument)



year,meanpm2.5,meanTEMP,meanIws,meanHUMID
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2010,96.76793,11.64800,28.32749,49.68798
2011,90.27650,12.52983,26.18118,47.97093
2012,85.30036,11.97797,23.87741,50.19599
2013,101.34487,12.50381,21.16345,44.51419
2014,96.54557,13.65571,19.08645,41.96638


`summarise()` ungrouping output (override with `.groups` argument)



month,meanpm2.5,meanTEMP,meanIws,meanHUMID
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,106.85961,-4.540754,36.32164,42.54578
2,127.27723,-1.477624,17.77069,42.89109
3,94.33321,6.080593,26.93827,25.00542
4,78.36613,13.994879,34.87003,22.09217
5,77.05971,21.270647,26.91528,29.80515
6,91.62495,24.534515,11.44572,57.99003
7,92.69364,27.192736,13.56171,67.00108
8,71.95997,25.746605,12.03299,67.66798
9,78.36357,20.061578,10.90076,64.34366
